In [1]:
import os
import sys
import math
import time
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import lib.params as params
import lib.network as network
import lib.utils as utils
from datetime import datetime

In [2]:
%load_ext autoreload
%autoreload 2
#read params
with open("config/train.params") as f:
    learning_rate=float(params.read_param(f.readline()))
    batch_size=int(params.read_param(f.readline()))
    epoch=int(params.read_param(f.readline()))
   
    print("training with a learning rate of {} for {} epochs with batchs of size {}".format(learning_rate,epoch,batch_size))
    
data_all=np.load("out/all_data.npy")  
label_all=np.load("out/all_labels.npy")
# print(data_all.shape,label_all.shape)

training with a learning rate of 0.1 for 1 epochs with batchs of size 3


In [7]:
%matplotlib inline
print("training")
net=network.R2N2(learning_rate)
model_dir="out/model_{}_{}_{} {}".format(learning_rate,epoch,batch_size,datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

# train network
loss_all=[]
acc_all=[]
N=len(data_all)
for e in range(epoch):
    start_time=time.time()
    perm=np.random.permutation(N)
    data_all=data_all[perm]
    label_all=label_all[perm]
    split_size=math.ceil(N/batch_size)
    data_batchs=np.array_split(data_all,split_size)
    label_batchs=np.array_split(label_all,split_size)
    
    print("epoch %d starts"%(e))
    save_dir="{}/epoch_{:03d}".format(model_dir,e)
    if not os.path.isdir(save_dir):
            os.makedirs(save_dir)
    loss_epoch=[]
    acc_epoch=[]
    batch_number=0
    for data,label in zip(data_batchs,label_batchs):
        fd={net.X:data, net.Y: label}
#         train_step=net.train_step(fd)
#         net.save_hidden_state(save_dir,fd)
        loss_epoch.append(net.final_encoder_state.eval(fd))
#         acc_epoch.append(net.mean_accuracy.eval(fd))
        print("epoch_{:03d}-batch_{:03d}: loss={}".format(e,batch_number,loss_epoch[-1]))
        batch_number+=1
        sys.exit()
        
            
    loss_all.append(loss_epoch)
    acc_all.append(acc_epoch)
    net.save(save_dir,loss_all,acc_all)
    print("epoch %d took %d seconds"%(e,time.time()-start_time))

training
epoch 0 starts
epoch_000-batch_000: loss=[[[-0.07121372 -1.3980571  -0.39468667 ..., -7.3202877   4.08890915
    0.45083928]
  [ 0.03615016 -1.07557905  0.28759524 ..., -5.94372845  4.46613932
    0.18392783]
  [ 0.23831642 -2.23874879 -0.32606506 ..., -6.57622528  4.95889091
    0.10449296]
  ..., 
  [ 0.04859376 -1.28883755  0.07362787 ..., -6.28833342  3.95998311
   -0.72899163]
  [ 0.49004942 -2.12506771  0.38724765 ..., -7.22942114  4.34837818
    0.80953276]
  [ 1.10244656 -2.20582247  0.46695802 ..., -4.55524778  5.05661011
    0.29440206]]

 [[ 1.48693013 -2.74202633 -0.64910406 ..., -8.46296024  4.62104511
   -0.19534856]
  [ 0.46266735 -1.39381409 -0.66477501 ..., -7.64242744  3.26437449
    0.53357023]
  [ 1.41221774 -2.29748201 -0.72554696 ..., -8.2671299   3.19469261
   -0.39396852]
  ..., 
  [ 2.00855613 -2.64269686 -0.01028085 ..., -8.05302048  3.32103491
   -0.58228397]
  [ 1.53044963 -1.91918111 -0.74095398 ..., -7.60092258  4.440588
   -0.48069945]
  [ 1.3476

SystemExit: 

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
